# UTD-MHAD Inertial Loader Exploration

Inspect the raw (non-upsampled) inertial clips using `src/datasets/utd_loader.py` and print simple dataset statistics.

In [10]:
from pathlib import Path
import numpy as np
import torch

from src.datasets.utd_loader import UTDMHADInertialDataset

DATA_ROOT = Path.cwd() / '..' / 'datasets' / 'UTD_MHAD'
print(f'Data root: {DATA_ROOT}')

Data root: /Users/parham/Desktop/The Semester/code/tests/../datasets/UTD_MHAD


In [11]:
dataset = UTDMHADInertialDataset(
    root=DATA_ROOT,
    subjects=['s1'],
    acc_file_suffix='inertial.npy',
    dtype=torch.float32,
)
print(f"Loaded {len(dataset)} clips from subjects: {dataset.subjects}")
print(f"Number of unique actions: {len(dataset.action_to_idx)}")
print('Action to index mapping (first 5):')
for action, idx in list(dataset.action_to_idx.items())[:5]:
    print(f'  {action} -> {idx}')

Loaded 83 clips from subjects: ['s1']
Number of unique actions: 21
Action to index mapping (first 5):
  a1 -> 0
  a10 -> 1
  a11 -> 2
  a12 -> 3
  a13 -> 4


In [12]:
sample_acc, sample_label, sample_meta = dataset[0]
print('Sample path:', sample_meta['path'])
print('Action label:', sample_meta['action'], 'mapped to index', sample_label)
print('Tensor shape (C, T):', tuple(sample_acc.shape))
print('First 5 timesteps (channel 0):', sample_acc[0, :5])

Sample path: /Users/parham/Desktop/The Semester/code/tests/../datasets/UTD_MHAD/s1/a10_s1_t1_inertial.npy
Action label: a10 mapped to index 1
Tensor shape (C, T): (6, 192)
First 5 timesteps (channel 0): tensor([-0.9517, -0.9485, -0.9404, -0.9448, -0.9546])


In [13]:
lengths = []
subjects = set()
actions = set()
for _, _, meta in dataset:
    lengths.append(meta['length'])
    subjects.add(meta['subject'])
    actions.add(meta['action'])

lengths = np.asarray(lengths)
print(f'Total clips scanned: {len(lengths)}')
print(f'Subjects covered: {sorted(subjects)}')
print(f'Unique actions: {sorted(actions)}')
print('Length stats (timesteps):')
print(f'  min:  {lengths.min()}')
print(f'  max:  {lengths.max()}')
print(f'  mean: {lengths.mean():.2f}')
print(f'  median: {np.median(lengths):.2f}')

Total clips scanned: 83
Subjects covered: ['s1']
Unique actions: ['a1', 'a10', 'a11', 'a12', 'a13', 'a14', 'a15', 'a16', 'a17', 'a18', 'a19', 'a2', 'a20', 'a21', 'a3', 'a4', 'a5', 'a6', 'a7', 'a8', 'a9']
Length stats (timesteps):
  min:  139
  max:  251
  mean: 186.13
  median: 180.00


In [14]:
from collections import Counter

length_buckets = Counter()
for L in lengths:
    bucket = int(np.floor(L / 25) * 25)
    length_buckets[bucket] += 1

print('Length histogram (bucketed by 25 samples):')
for bucket in sorted(length_buckets):
    print(f'{bucket:3d}-{bucket+24:3d}: {length_buckets[bucket]} clips')

Length histogram (bucketed by 25 samples):
125-149: 4 clips
150-174: 28 clips
175-199: 25 clips
200-224: 16 clips
225-249: 9 clips
250-274: 1 clips


## All subjects statistic

In [15]:
dataset_all = UTDMHADInertialDataset(
    root=DATA_ROOT,
    subjects=None,
    acc_file_suffix='inertial.npy',
    dtype=torch.float32,
)
lengths_all = np.asarray([meta['length'] for _, _, meta in dataset_all])
print(f'Total clips (all subjects): {len(lengths_all)}')
print('Global length stats:')
print(f'  min:  {lengths_all.min()}')
print(f'  max:  {lengths_all.max()}')
print(f'  mean: {lengths_all.mean():.2f}')

Total clips (all subjects): 671
Global length stats:
  min:  107
  max:  326
  mean: 178.68
